<a href="https://colab.research.google.com/github/anishaluthra/root_cause_finder/blob/main/basic_RAG_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

nvidia-smi

Thu Jan  9 12:17:18 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# @title Install necessary packages
%%bash

pip install haystack-ai accelerate "sentence-transformers>=3.0.0" "datasets>=2.6.1" PyPDF2 pydantic farm-haystack[all]

  Using cached haystack_ai-2.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached farm_haystack-1.26.4-py3-none-any.whl.metadata (31 kB)
  Using cached haystack_experimental-0.4.0-py3-none-any.whl.metadata (16 kB)
  Using cached lazy_imports-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached posthog-3.7.5-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached boilerpy3-1.0.7-py3-none-any.whl.metadata (5.8 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached lazy_imports-0.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached prompthub_p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires pydantic>=2.7.0, but you have pydantic 1.10.20 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
google-genai 0.3.0 requires pydantic<3.0.0dev,>=2.0.0, but you have pydantic 1.10.20 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.
langchain 0.3.14 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.20 which is incompatible.
langchain-core 0.3.29 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.20 which is incompatible.
wandb 0.19.1 requires pydantic<3,>=2.6, but you have pydantic 1.10.20 which is incompatible.


In [4]:
# Install necessary libraries
#!pip install farm-haystack[all] PyPDF2

In [5]:
import os
from PyPDF2 import PdfReader
from datasets import load_dataset
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your folder containing documents on Google Drive
folder_path = "/content/drive/My Drive/root_cause_investigator/migraine_publications"

# Read PDF files and extract text
documents = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        try:
            reader = PdfReader(file_path)
            content = ""
            for page in reader.pages:
                content += page.extract_text()  # Extract text from each page
            documents.append(Document(content=content, meta={"filename": filename}))
        except Exception as e:
            print(f"Error reading {filename}: {e}")

Mounted at /content/drive


In [6]:
# Limit to the first 5 documents
for i, doc in enumerate(documents[:5]):  # Slice the first 5 documents
    content_preview = doc.content[:2000]  # Get the first 2000 characters
    print(f"Document {i+1}: {doc.meta['filename']}")
    print(content_preview)
    print("\n" + "="*100 + "\n")  # Separator for better readability

Document 1: The Participant in Migraine and Recent Advancements in Translational Research.pdf
/gid00030/gid00035/gid00032/gid00030/gid00038/gid00001/gid00033/gid00042/gid00045 /gid00001
/gid00048/gid00043/gid00031/gid00028/gid00047/gid00032/gid00046Citation: Spekker, E.; Tanaka, M.;
Szabó, Á.; Vécsei, L. Neurogenic
Inflammation: The Participant in
Migraine and Recent Advancements in
Translational Research. Biomedicines
2022 ,10, 76. https://doi.org/
10.3390/biomedicines10010076
Academic Editor: Marco Segatto
Received: 2 November 2021
Accepted: 27 December 2021
Published: 30 December 2021
Publisher’s Note: MDPI stays neutral
with regard to jurisdictional claims in
published maps and institutional afﬁl-
iations.
Copyright: © 2021 by the authors.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed under the terms and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
biomedicines
Review
Neuroge

In [7]:
model = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

document_store = InMemoryDocumentStore()

indexing_pipeline = Pipeline()

indexing_pipeline.add_component(instance=SentenceTransformersDocumentEmbedder(model=model), name="embedder")
indexing_pipeline.add_component(instance=DocumentWriter(document_store=document_store), name="writer")
indexing_pipeline.connect("embedder.documents", "writer.documents")

indexing_pipeline.run({"documents": documents})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

{'writer': {'documents_written': 37}}

In [8]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersTextEmbedder


retriever = InMemoryEmbeddingRetriever(document_store=document_store)
reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()

extractive_qa_pipeline.add_component(instance=SentenceTransformersTextEmbedder(model=model), name="embedder")
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")

extractive_qa_pipeline.connect("embedder.embedding", "retriever.query_embedding")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - reader: ExtractiveReader
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reader.documents (List[Document])

In [11]:
query = "What are the root causes of migraines?"
extractive_qa_pipeline.run(
    data={"embedder": {"text": query}, "retriever": {"top_k": 5}, "reader": {"query": query, "top_k": 5}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reader': {'answers': [ExtractedAnswer(query='What are the root causes of migraines?', score=0.7798208594322205, data='Single-nucleotide polymorphism alleles in theinsulin receptor gene', document=Document(id=d1f562454a7163b46032ec25e4fe4a6a7476317cbd920eab06cd6650547dd066, content: 'PATHOPHYSIOLOGY OF MIGRAINE: A DISORDER
   OF SENSORY PROCESSING
   Peter J. Goadsby, Philip R. Holland, M...', meta: {'filename': 'Pathophysiology of Migraine A Disorder of Sensory Processing.pdf'}, score: 25.30805336616566), context=None, document_offset=ExtractedAnswer.Span(start=377782, end=377848), context_offset=None, meta={}),
   ExtractedAnswer(query='What are the root causes of migraines?', score=0.7766060829162598, data='Fabric softener, sweat, socks, coffee, excrement, vomit, and\nanimals', document=Document(id=5aeeee88315258d9578c218ebb1e8eba464eaebbc8bdf6bb5278a47e5241a5e2, content: 'Page 1/14
   Classi cation of odorant substances that trigger
   migraine attacks: a cross-sectional study...'

In [12]:
query = "What are treament options for migraines?"
extractive_qa_pipeline.run(
    data={"embedder": {"text": query}, "retriever": {"top_k": 5}, "reader": {"query": query, "top_k": 5}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reader': {'answers': [ExtractedAnswer(query='What are treament options for migraines?', score=0.7690701484680176, data='conventional oral medications', document=Document(id=80f078d426de69c0041aecd46f12158592c70a6d143b22f43fb76bd58cad34b2, content: 'Citation: Nisar, A.; Ahmed, Z.; Yuan,
   H. Novel Therapeutic Targets for
   Migraine. Biomedicines 2023 ,...', meta: {'filename': 'biomedicines-11-00569.pdf'}, score: 21.98535782161427), context=None, document_offset=ExtractedAnswer.Span(start=4781, end=4810), context_offset=None, meta={}),
   ExtractedAnswer(query='What are treament options for migraines?', score=0.7560072541236877, data='antidepressants (amitriptyline, venlafaxine); beta-blockers (atenolol, nadolol); and angiotensin receptor blockers (candesartan).', document=Document(id=c8cbf9083b53e99e119f1488ff68a0c15cc2c58d37e829b3d74a8ce0926cb1e4, content: '1AHS Consensus Statement
   The American Headache Society Position Statement On 
   Integrating New Migrai...', meta: {'filena